In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 1.8MB 15.0MB/s 
     |████████████████████████████████| 1.2MB 40.8MB/s 
     |████████████████████████████████| 2.9MB 46.6MB/s 
     |████████████████████████████████| 890kB 43.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.1.2-cp36-none-any.whl size=103068 sha256=df8677b213be4226b84c06d30f0a7359e53a5173f40c2fe00e03248758cd062f
  Stored in directory: /root/.cache/pip/wheels/3d/33/d1/5703dd56199c09d4a1b41e0c07fb4e7765a84d787cbdc48ac3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6e16db766764d5583fc5e907d0e625ebd9148a0110aec2f1fd156621f5969a8b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
from sentence_transformers import SentenceTransformer

Roberta_model=SentenceTransformer('paraphrase-distilroberta-base-v1')

100%|██████████| 306M/306M [00:19<00:00, 15.9MB/s]


In [ ]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

In [ ]:
val=Roberta_model.encode(sentences)

In [ ]:
val.shape

(3, 768)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# our bench mark data set
df=pd.read_csv("/content/drive/MyDrive/Data/quora duplicate identification/train.csv")
df.head()



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df=df.dropna()

In [ ]:
questions=list(df['question1'])+(list(df['question2']))

In [ ]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
!pip install contractions

     |████████████████████████████████| 327kB 7.9MB/s 
     |████████████████████████████████| 245kB 14.1MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp36-cp36m-linux_x86_64.whl size=84340 sha256=f107560b23892ecd384a269beb972f447dbd0a35d19567f04deaef3c6268db3d
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick


In [ ]:
from data_cleaner import basic_clean,normalization

questions=[basic_clean(text) for text in questions[:1000]]
questions=[normalization(text) for text in questions[:1000]]

In [ ]:
q_embedding=Roberta_model.encode(questions)
q_embedding

array([[-0.16360234, -0.17930153,  0.14914513, ...,  0.1338934 ,
        -0.3437827 , -0.00413765],
       [-0.01563864,  0.05274884,  0.05677265, ..., -0.34794858,
         0.1469091 ,  0.2069199 ],
       [ 0.35172418,  0.3334732 ,  0.0106143 , ...,  0.11238978,
         0.12639314,  0.1665183 ],
       ...,
       [ 0.31753832,  0.44577074,  0.05151366, ..., -0.33203867,
        -0.12375947, -0.09089129],
       [-0.11348309, -0.06439368, -0.07496834, ..., -0.00777846,
         0.2978264 ,  0.03034527],
       [ 0.16686626,  0.03655932, -0.10955131, ...,  0.17079645,
         0.07372221, -0.44336432]], dtype=float32)

In [ ]:
q_embedding[0],q_embedding

((1000, 768),
 array([-1.63602337e-01, -1.79301530e-01,  1.49145126e-01,  1.65309042e-01,
        -4.57854569e-02, -2.21105888e-01, -1.45940082e-02, -7.79695017e-03,
        -4.39864844e-02, -7.57869408e-02, -4.20561619e-02, -1.70047525e-02,
        -7.22681778e-03,  4.97855023e-02, -2.22555384e-01,  4.62932557e-01,
         4.38784212e-01, -4.77449745e-01,  2.33722761e-01,  1.61527514e-01,
        -8.71991962e-02,  2.12490916e-01, -1.06336810e-01,  1.59264192e-01,
        -7.35741556e-02,  2.82213688e-01, -4.93484102e-02, -2.86643147e-01,
        -3.56438607e-01, -2.66299486e-01, -1.65006682e-01,  7.40244985e-02,
         1.48846760e-01, -9.05571282e-02,  3.69553626e-01,  1.89232841e-01,
         1.92145661e-01, -1.10085994e-01,  2.16255859e-01, -1.60995618e-01,
        -7.28716135e-01, -8.61847699e-02, -2.53772289e-01, -3.10930535e-02,
         9.00732353e-02, -2.20424682e-01,  1.63560748e-01, -1.57161221e-01,
        -4.68696561e-03,  1.17315548e-02, -2.52648085e-01,  5.21022826e-02

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query=str(input())
encoding=Roberta_model.encode(query)
score=max([cosine_similarity([encoding],[embedding]) for embedding in q_embedding])

print(score)

Who is the PM of india
[[0.42471898]]


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([[1,2,3,12,12]],[[1,2,212,112,2]])


array([[0.48211858]])

In [ ]:
for embedding in q_embedding:
  print(embedding.shape)
  break

(768,)
